In [82]:
## 1. 기본
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 2. 훈련 검증용 데이터 분리
from sklearn.model_selection import train_test_split

# 3. 교차검증
from sklearn.model_selection import cross_validate

# 4. 정규화
from sklearn.preprocessing import StandardScaler

# 5. 분류모델 구축
# 1) ★K-최근접이웃(KNN, 이웃값 조정)
from sklearn.neighbors import KNeighborsClassifier 
# 2) ★로지스틱 회귀모델
from sklearn.linear_model import LogisticRegression 
# 3) 확률적 경사 하강법
from sklearn.linear_model import SGDClassifier 
# 4) 결정트리
from sklearn.tree import DecisionTreeClassifier
# 5) 앙상블 모델
from sklearn.ensemble import RandomForestClassifier # ★랜덤포레스트
from sklearn.ensemble import ExtraTreesClassifier # 엑스트라 트리
from sklearn.ensemble import GradientBoostingClassifier # ★그레디언트 부스팅
from sklearn.ensemble import HistGradientBoostingClassifier # ★히스토그램 기반 그레디언트 부스팅

# from xgboost import XGBClassifier # xgboost
# from lightgbm import LGBMClassifier # lightgbm

## 모듈 import
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist
from keras.optimizers import RMSprop
from keras.losses import CategoricalCrossentropy

In [ ]:
import warnings
warnings.filter

In [4]:
import numpy as np
import pandas as pd

In [5]:
import tensorflow
from tensorflow import keras
from keras.models import Model,Sequential
from keras.layers import Dense,Flatten,BatchNormalization
from keras import initializers
from keras.utils.np_utils import to_categorical
from keras.losses import CategoricalCrossentropy

In [42]:
#### 옵티마이저들 적재(주로 이정도만씀)
from keras.optimizers import Adam,RMSprop,SGD,Adagrad,Nadam

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
#train_data=pd.read_clipboard()

In [49]:
train_data = pd.read_csv("./data/titanic.csv")

In [50]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     888 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [51]:
## 데이터 정제
train_data=train_data.iloc[:,1:]
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  888 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [52]:
y=train_data.iloc[:,0].values
X=train_data.iloc[:,1:]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  888 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [53]:
## 결측값 처리(Age,Embarked,Cabin)
# Cabin은? 결측값이 많지만 '미상'으로 처리(그게 영향있을 수 있으니까)

In [54]:
X.Cabin.value_counts()
X.Cabin.unique().size

148

In [56]:
X.Cabin.str.slice(0,1).value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [57]:
X.Cabin.fillna('N',inplace=True)
X.Cabin.str.slice(0,1)

0      N
1      C
2      N
3      C
4      N
      ..
886    N
887    B
888    N
889    C
890    N
Name: Cabin, Length: 891, dtype: object

In [58]:
X['new_cabin']=X.Cabin.str.slice(0,1)

In [59]:
X.drop(['Cabin'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        714 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   888 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [60]:
X.Embarked.value_counts()

S    644
C    168
Q     76
Name: Embarked, dtype: int64

In [61]:
X.Embarked.fillna('S',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        714 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   891 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [62]:
# 성별 평균 나이로 채우는 게 좋겠다
X.loc[X.Age.isna(),['Name','Age']]

,Name,Age
5,"Moran, Mr. James",NaN
17,"Williams, Mr. Charles Eugene",NaN
19,"Masselmani, Mrs. Fatima",NaN
26,"Emir, Mr. Farred Chehab",NaN
28,"O'Dwyer, Miss. Ellen ""Nellie""",NaN
...,...,...
859,"Razi, Mr. Raihed",NaN
863,"Sage, Miss. Dorothy Edith ""Dolly""",NaN
868,"van Melkebeke, Mr. Philemon",NaN
878,"Laleff, Mr. Kristo",NaN


In [63]:
X.Age[(X.Age.isna()) & (X.Name.str.contains('Mr.'))]= X.Age[X.Name.str.contains('Mr.')].mean()
X.Age[(X.Age.isna()) & (X.Name.str.contains('Mrs.'))]= X.Age[X.Name.str.contains('Mrs.')].mean()
X.Age[(X.Age.isna()) & (X.Name.str.contains('Miss.'))]= X.Age[X.Name.str.contains('Miss.')].mean()

C:\Users\admin\AppData\Local\Temp\ipykernel_8136\284447569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Age[(X.Age.isna()) & (X.Name.str.contains('Mr.'))]= X.Age[X.Name.str.contains('Mr.')].mean()
C:\Users\admin\AppData\Local\Temp\ipykernel_8136\284447569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Age[(X.Age.isna()) & (X.Name.str.contains('Mrs.'))]= X.Age[X.Name.str.contains('Mrs.')].mean()
C:\Users\admin\AppData\Local\Temp\ipykernel_8136\284447569.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

In [64]:
# 5명 안채워짐(아마 child)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        886 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   891 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [65]:
X.loc[X.Age.isna(),['Name','Age','Sex']]

,Name,Age,Sex
65,"Moubarek, Master. Gerios",NaN,male
159,"Sage, Master. Thomas Henry",NaN,male
176,"Lefebre, Master. Henry Forbes",NaN,male
709,"Moubarek, Master. Halim Gonios (""William George"")",NaN,male
766,"Brewe, Dr. Arthur Jackson",NaN,male


In [66]:
X.Age[X.Age.isna()]=X.Age[X.Name.str.contains('Mr')|X.Name.str.contains('Miss')|X.Name.str.contains('Mrs')].mean()

C:\Users\admin\AppData\Local\Temp\ipykernel_8136\1316724870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Age[X.Age.isna()]=X.Age[X.Name.str.contains('Mr')|X.Name.str.contains('Miss')|X.Name.str.contains('Mrs')].mean()


In [67]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        891 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   891 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [68]:
# Embarked

In [69]:
## 그냥 get_dummies 로 하면 데이터가 새로 들어왔을 때 문제가 됨
## 정수인코딩->원핫 해줘야한다

In [70]:
# 정수인코딩 모듈 임포트
from sklearn.preprocessing import LabelEncoder

In [71]:
le_embarked=LabelEncoder()
le_embarked.fit(X.Embarked)

LabelEncoder()

In [72]:
# fit_transform으로 하지 말라고함
X['new_embarked']=le_embarked.transform(X.Embarked)    

In [73]:
le_embarked.transform(np.array(['S']))

array([2])

In [74]:
X.drop(['Embarked'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Sex           891 non-null    object 
 3   Age           891 non-null    float64
 4   SibSp         891 non-null    int64  
 5   Parch         891 non-null    int64  
 6   Ticket        891 non-null    object 
 7   Fare          891 non-null    float64
 8   new_cabin     891 non-null    object 
 9   new_embarked  891 non-null    int32  
dtypes: float64(2), int32(1), int64(3), object(4)
memory usage: 66.3+ KB


In [75]:
## object들 인코딩

In [76]:
# Sex는 0,1 둘뿐이므로 np.where로 하면됨
X['gender']=np.where(X.Sex=='male',0,1)

In [77]:
X.drop(['Sex'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Age           891 non-null    float64
 3   SibSp         891 non-null    int64  
 4   Parch         891 non-null    int64  
 5   Ticket        891 non-null    object 
 6   Fare          891 non-null    float64
 7   new_cabin     891 non-null    object 
 8   new_embarked  891 non-null    int32  
 9   gender        891 non-null    int32  
dtypes: float64(2), int32(2), int64(3), object(3)
memory usage: 62.8+ KB


In [78]:
le_cabin=LabelEncoder()
le_cabin.fit(X.new_cabin)
X.new_cabin=le_cabin.transform(X.new_cabin)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Age           891 non-null    float64
 3   SibSp         891 non-null    int64  
 4   Parch         891 non-null    int64  
 5   Ticket        891 non-null    object 
 6   Fare          891 non-null    float64
 7   new_cabin     891 non-null    int32  
 8   new_embarked  891 non-null    int32  
 9   gender        891 non-null    int32  
dtypes: float64(2), int32(3), int64(3), object(2)
memory usage: 59.3+ KB


In [79]:
# 티켓은 종류가 너무 많음-->날림
X.Ticket.value_counts().size

681

In [80]:
X.drop(['Ticket'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Age           891 non-null    float64
 3   SibSp         891 non-null    int64  
 4   Parch         891 non-null    int64  
 5   Fare          891 non-null    float64
 6   new_cabin     891 non-null    int32  
 7   new_embarked  891 non-null    int32  
 8   gender        891 non-null    int32  
dtypes: float64(2), int32(3), int64(3), object(1)
memory usage: 52.3+ KB


In [83]:
X.drop(['Name'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Age           891 non-null    float64
 2   SibSp         891 non-null    int64  
 3   Parch         891 non-null    int64  
 4   Fare          891 non-null    float64
 5   new_cabin     891 non-null    int32  
 6   new_embarked  891 non-null    int32  
 7   gender        891 non-null    int32  
dtypes: float64(2), int32(3), int64(3)
memory usage: 45.4 KB


In [ ]:
### X,y 데이터로 Start

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [95]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(668, 8)
(668,)
(223, 8)
(223,)


In [96]:
hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)

print("히스토그램 기반 그레디언트 부스팅")
print("훈련 :",np.mean(scores["train_score"]), "/" , "테스트 :", np.mean(scores["test_score"]))

print("---------------------------------------------------------")

gb = GradientBoostingClassifier(random_state=42)

scores = cross_validate(gb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)

print("그레디언트 부스팅")
print("훈련 :",np.mean(scores["train_score"]), "/" , "테스트 :", np.mean(scores["test_score"]))

print("---------------------------------------------------------")
rt = RandomForestClassifier(random_state=42)

scores = cross_validate(rt, X_train, y_train,
                        return_train_score=True, n_jobs=-1)

print("랜덤포레스트")
print("훈련 :",np.mean(scores["train_score"]), "/" , "테스트 :", np.mean(scores["test_score"]))

히스토그램 기반 그레디언트 부스팅
훈련 : 0.9573376737022647 / 테스트 : 0.8368421052631578
---------------------------------------------------------
그레디언트 부스팅
훈련 : 0.9240288424516084 / 테스트 : 0.8383121984064641
---------------------------------------------------------
랜덤포레스트
훈련 : 0.9910203367286219 / 테스트 : 0.8128605094826618


In [97]:
### 정규화

ss = StandardScaler()
ss.fit(X_train)
train_scaled = ss.transform(X_train)
test_scaled = ss.transform(y_train)

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print("KNN")
print("훈련 :", kn.score(train_scaled, train_target), "/", "테스트 :", kn.score(test_scaled, test_target))

print("---------------------------------------------------------")

lr = LogisticRegression()
lr.fit(train_scaled, train_target)

print("로지스틱 회귀분석")
print("훈련 :", lr.score(train_scaled, train_target), "/", "테스트 :", lr.score(test_scaled, test_target))

print("---------------------------------------------------------")

dt = DecisionTreeClassifier(max_depth=6, random_state=42)

dt.fit(train_scaled, train_target)

print("결정트리")
print("훈련 :", dt.score(train_scaled, train_target), "/", "테스트 :", dt.score(test_scaled, test_target))

c:\Users\admin\anaconda3\envs\keras1\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0